# Jupyter Notebook UI to graph your speed trace (with opto) data!

Version 1.3 - Joseph Liang, Rankin Lab
Updated:
1. Upgraded folder path selection application
2. Upgraded dataset management (less moving parts for end-user)
3. output changed from tif -> png

## Known bug: Step 2 an empty windows displays in Mac. May also apply to linux/windows.

## Beginner Essentials:
1. Shift-Enter to run each cell. After you run, you should see an output "done step #". If not, an error has occured
2. When inputting your own code/revising the code, make sure you close all your quotation marks '' and brackets (), [], {}.
3. Don't leave any commas (,) hanging! (make sure an object always follows a comma. If there is nothing after a comma, remove the comma!
4. Learning to code? Each line of code is annotated to help you understand how this code works!

## 3. Run all cells/steps sequentially, even the ones that do not require input

## Steps that require input: #3, #6.1, #7

# 1. Importing Packages Required (No input required, just run)

In [ ]:
import pandas as pd #<- package used to import and organize data
import numpy as np #<- package used to import and organize data
import seaborn as sns #<- package used to plot graphs
from matplotlib import pyplot as plt #<- package used to plot graphs
import os #<- package used to work with system filepaths
from ipywidgets import widgets #<- widget tool to generate button
from IPython.display import display #<- displays button
from tkinter import Tk, filedialog #<- Tkinter is a GUI package
print("done step 1")

# 2. Pick filepath (just run and click button)

Run the following cell and clicke the button 'Select Folder' to pick a filepath.

## Important: Later on, this script uses the total file path for each file to import and group data. That means if your folder has whatever your strain is named, the script will not work.

(ex. if your folder has "N2" in it this script sees all files inside this folder as having the "N2" search key)

## An easy fix is to just rename your folder to something else (make your strains lower-case, or just have the date)

In [ ]:
### Select Folder App - After you run, you can select your folder for filepath
button = widgets.Button(description = 'Select Folder') #<- creates a button variable
display(button) #<- displays the button in output

def select_folder(b): #<- defines an action. This action requires a variable, so an arbitrary one 'b' is there
    global folder_path #<- sets a variable as a global variable, not just within this action
    #Tk().withdraw() #<- Tkinter likes to create annoying empty windows. This removes them
    folder_path = filedialog.askdirectory() #<- Opens up a file explorer window, and determines folder path
    #Tk().update() #<- below
    #Tk().destroy() #<- this and the line above it removes the file explorer window after a selection is made
    print(folder_path) #<- this helps confirm that this action was performed
    print('done step 2')
button.on_click(select_folder)

# 3. User Defined Variables (Add input here)

Here, we add some constants to help you blaze through this code.

## 3.1: Setting time bins
pretty self explanatory

## 3.2: Setting light flashes (for visualization of light pulses in opto experiments)
Here you are generating a separate dataframe to draw out light bars on your graph to visualize your light pulses. This will take a bit of elementary-level math. Think of it as drawing (connecting-the-dots style) rectangles that are 0.5 high strung-together. Start at (0,0), then (x1,0), then (x1, y1) ... etc

____|-----|______|-----|____ etc.  

Once you have that written down, put all the y's together in one dataframe, and the x's together in the other one..

## 3.3: Setting view range for your graph
Top, bottom = y axis view range
left, right = x axis view range



In [ ]:
# Setting 0.5s Bins
bins = np.linspace(0,1200,2400) #<- np.linspace(start, end, steps in between)

## If you change only something below this line, you can skip straight to step {Graphing} after.

# Light Flashes (for optogenetic/ChR2 experiments), if you are not doing Opto, disregard and run as is
# lighton = np.arange(600, 890, 70) # where the bars start (start of first pulse, end of last pulse, time between)
# lightoff = np.arange(610, 900, 70) # where the bars end (end of first pulse, end of last pulse + 10s, time between)
# lighton = np.arange(300, 940, 70) # where the bars start (start of first pulse, end of last pulse, time between)
# lightoff = np.arange(310, 950, 70) # where the bars end (end of first pulse, end of last pulse + 10s, time between)
# lighton = np.arange(600, 670, 70) #long pulse new
# lightoff = np.arange(660, 670, 70) #long pulse new
lighton = np.arange(300, 370, 70) #long pulse new
lightoff = np.arange(360, 370, 70) #long pulse new
# lighton = np.arange(200, 270, 70) #old long pulse
# lightoff = np.arange(260, 270, 70) #old long pulse

# create the dictionary
d1 = dict( zip(lighton, lightoff) )
# check
print("d1 = {}".format(d1))
# turn the dictionary into an ITERABLE list of tuples
d2 = list(d1.items()) 



print("done step 3")

# 4. Construct filelist from folder path (No input required, just run)

In [ ]:
#folder_path = '/Users/Joseph/Desktop/lrk1_ATR_NoATR_08_15_2019'
print(folder_path)
os.chdir(folder_path) #<- setting your working directory so that your images will be saved here

filelist = list() #<- empty list
for root, dirs, files in os.walk(folder_path): #<- this for loop goes through your folder 
    for name in files:
        if name.endswith('.dat'): #<- and takes out all files with a .dat (file that contains your data)
            filepath = os.path.join(root, name) #<- Notes down the file path of each data file
            filelist.append(filepath) #<- saves it into the list
    
print(filelist)
print('done step 4')

# 5. Process Data Function (No input required, just run)

In [ ]:
def ProcessData(strain): #<- an example of a user-defined function
    strain_filelist = []  #<- empty list
    for i in filelist: #<- goes through the list and filters for keyword
        if strain in i:
            strain_filelist.append(i)
    Strain_N = len(strain_filelist) #<- N per strain, or number of plates
    print(f'This Strain has {Strain_N} plates') #<- prints out number of plates per strain
    for i, f in enumerate(strain_filelist, start=1): #<- a progress bar function integrated into data import
        N = len(strain_filelist)
        statement = f"Processing {i} of {N} plates" #<- progress bar statement
        DF_Read = pd.read_csv(f, sep=' ', skiprows = 0, header = None) #<- imports and cleans data
        if i==1:
            DF_Total = DF_Read
        else:
            DF_Total = pd.concat((DF_Total, DF_Read), ignore_index = False)  #<- imports and cleans data
        print(statement)
    DF_Total = DF_Total.dropna(axis = 1) #<- more data cleaning
    DF_Total = DF_Total.rename( #<- more data cleaning
                {0:'time',
                1:'speed'}, axis=1)
    #0.5s Bins
    Bins = [float(i) for i in bins] #<- adds time bins into the data frame
    DF_Total['time_bin'] = pd.cut(DF_Total['time'], Bins, labels = Bins[1:])
    DF_Speed_DF = DF_Total[["time_bin", "speed"]].copy()
    
    return{
            'N': Strain_N,
            'Confirm':DF_Total,
            'Speed_DF': DF_Speed_DF, #<- this is the finished product we will use for analysis
            'Filelist':strain_filelist} 

print('done step 5')

# 6.1 Processing Data (Please Read, Input Here)

Here is the hardest part - From your naming convention, you need to pick a unique identifier for each group.

This means that all of names of your files for that strain should have that in common but is not commone with across all other files! If you did a good job naming your files and following a good naming convention, this should be easy.

## Be careful and really look hard in your naming structure. Note you want an unique identifier in the entire file path for the same group of files. An easy mistake is to have the strain name in the folder name, in this case if you use your strain name as a keyword it would include all files in that folder!

For example, if all your N2 files have a certain pattern like "N2_5x4" in this following example:
'/Users/Joseph/Desktop/AVR14_10sISI_TapHab_0710_2019/N2/20190710_141740/N2_5x4_f94h20c_100s30x10s10s_C0710ab.trv'

Then you need to set that identifier for the strain keyword:
'Strain_1' = 'N2_5x4'

In the same example, if the identifier for your second strain is 'AVR14', then the N2 files will also be included, as this identifier can also be found in this file path.

## Depending on how many strains you are running for comparison, you may need to add/delete some lines.

You are not naming your data groups here, we have a step for that later.
## Here, you want to note down ALL the strains you have in the folder

If you have just 2 strains, add hashtags (#) in front of the lines you do not need.
If you need more strains, just add more lines, following the same format!

In [ ]:
#Format: Strain_# = 'unique_identifier'

### Make into dictionary
StrainNames = {
    'Strain_1' : 'VG923_ATR',   #<- each strain will be designated to a unique identifier here
    'Strain_2' : 'VG929_ATR',
#     'Strain_3' : 'VG929_ATR_',
#     'Strain_4' : 'VG928_NoATR_',
#     'Strain_5' : 'VG929_ATR_10',
    'Strain_6' : '',
    'Strain_7' : '',
    'Strain_8' : '',
    'Strain_9' : '',  #<- empty entries are for those hardcore trackers that tracking this many strains
    'Strain_10' : '',
    'Strain_11' : '',
    'Strain_12' : '',
    'Strain_13' : '',
    'Strain_14' : '',
    'Strain_15' : '',}
#...etc, etc

print('done step 6.1')

# 6.2 Process Data (just run this cell)

In [ ]:
DataLists = [0] #<- generates empty list. 0 is there to account for python's index starting at 0. 
# we want indexing to start at 1 (when I say #1 I want the first point, not the second point)

for s in StrainNames.values(): #<- goes through the dictionary in step 6.1 and processes data
    if not s == '':
        DataLists.append(ProcessData(s)['Speed_DF']) #<- appends all data into a list of dataframes
# print(DataLists[2])
print('done step 6.2')

# 7. Grouping Data and Naming (Optional: Add input here)

Here, you get to name your data groups/strain! Name your groups however you like under between the quotation marks for each strain.

For example: If your Strain1 is N2 and you wish for the group to be called N2,
your line should look like:

DataLists[x].assign(dataset = 'N2')

## Go back to step 6.1 to check which strain is which item on the DataLists.
In this example, the first item on DataLists is AQ2028_b.


## Remember: Put your name in quotes. (ex: 'N2' and not N2)

As default, the names are set to the unique identifier labels.

## Depending on the number of strains you are running the comparison, you may have to delete/add lines of code (following the same format). 
## Remember to add/delete commas too.

# If you want to change your groups, you do that here. 
For example, if you have 5 strains in your folder but only want to compare between 2 or 3 strains, designate that here and follow through with steps 6 and 7. Once you are done, come back to step 6 and change your groups again (You are going to have to change your graph titles for the second run-through though)!

In [ ]:
#to add a line, add a comma to the last line and press 'enter' to type into a new line
TotalConcatenated = pd.concat([
                            DataLists[1].assign(Strain = "Wildtype"),
                               DataLists[2].assign(Strain = "cat-2 (e1112)"),
#                                DataLists[3].assign(Strain = "lrk-1 (tm1898) + ATR"),
#                                DataLists[4].assign(Strain = "lrk-1 (tm1898) - ATR"),
#                                DataLists[5].assign(Strain = "cat-2 (e1112) + ATR")
#                                DataLists[6].assign(Strain = "cat-2 (e1112)")
                              ])
print(TotalConcatenated)
print('done step 7')

# 8.1 Graphing with Light Pulses (Skip to 7.2 if you are not using ChR2)

## Here, feel free to change the graph title and axis labels! Images are automatically saved in your folder with your data.

# Here, you may need to be patient - this code goes through millions of rows of data to plot your graph, so it may take a minute (or two.... or more...)

Palette:

Wildtype (ce314): (0.2980392156862745, 0.4470588235294118, 0.6901960784313725)

lrk-1: (0.8666666666666667, 0.5176470588235295, 0.3215686274509804)

vps-35 (hu68): (0.7686274509803922, 0.3058823529411765, 0.3215686274509804)

vps-35 (ok1880): (0.5058823529411764, 0.4470588235294118, 0.7019607843137254)

cat-2 (e1112): #8B008B

cat-2 (e1112) No ATR: #FF00FF

ce314 without ATR: #20B2AA (light see green)

ok530 with ATR: #8B4513 (saddle brown)

ok530 without ATR: #D2691E (chocolate)

control + ATR: "black"

control - ATR: "gray"

lrk-1_1 + ATR: "navy"

lrk-1_1 - ATR: "royalblue"

lrk-1_2 + ATR: "green"

lrk-1_2 - ATR: "lightgreen"

In [ ]:
# Setting viewing range for your graph
top = 0.35
bottom = 0
left = 250
right = 450

sns.set_context("talk")
plt.figure(linewidth = 2.5)#<- Make your figure lines THICCCC for clean display (posters)
plt.figure(figsize=(12,7))#<- set your figure size (width, length)
plt.gca().xaxis.grid(False)#<- gets rid of x-axis markers to make data look clean 
for tuple in d2:
    print("Creating bar: {}-{}".format(tuple[0], tuple[1]))
    plt.axvspan(xmin=tuple[0], xmax=tuple[1], facecolor="#99FFFF", alpha=0.2) # alpha controls how transparent the bars are  #<- the blue bars indicating light pulse
ax = sns.lineplot(x="time_bin", #<- Here we use seaborn as our graphing package. 
             y="speed", 
             data = TotalConcatenated,
             hue = 'Strain', #<- Here we use the extra column from step 6 to separate by group
             palette = [
                         "black",
#                  (0.2980392156862745, 0.4470588235294118, 0.6901960784313725),
#                         '#20B2AA',
#                         'green',
#                         'lightgreen',
                        '#8B008B'
#                         'royalblue',
#                         'green',
#                         '#D2691E'
#                         "lightgreen"
                       ]#<- Change colour palette here if you like
                 )
plt.xlabel("Time (s)")#<- X-axis title
plt.ylabel("Speed (mm/s)")#<- Y-Axis title
plt.title("")#<- Figure Title
plt.ylim(top = top) #<- setting the viewing range, from previous input in step 2
plt.ylim(bottom = bottom)
plt.xlim(left= left)
plt.xlim(right = right)
ax.legend(loc = 'upper right', fontsize = '13') #<- location of your legend
# plt.legend([],[], frameon=False)
# plt.tick_params(
#     axis='x',
#     which='both',
#     bottom=False,
#     top=False,
#     labelbottom=False)
# ax.legend().remove()
# plt.savefig(f'Speed_Trace_{left}to{right}s.png', format='png', dpi=450) #<- saves the figure to a tif file
plt.savefig(f'COMP_ATRONLY_Speed_Trace_{left}to{right}s_ATRNoATR.png', format='png', dpi=450) #<- saves the figure to a png file
plt.show()


# 8.2 Graphing without Light Pulse (Run this if you dont use ChR2)

In [ ]:
plt.figure(linewidth = 2.5)
plt.figure(figsize=(12,7))
plt.gca().xaxis.grid(False)
ax = sns.lineplot(x="time_bin", 
             y="speed", 
             data = TotalConcatenated,
             hue = 'dataset',
             palette = 'deep')
plt.xlabel("Time (s)")
plt.ylabel("Speed (mm/s)")
plt.title("Speed Trace")
plt.ylim(top = top)
plt.ylim(bottom = bottom)
plt.xlim(left= left)
plt.xlim(right = right)
ax.legend(loc = 'upper right', fontsize = '13')
plt.savefig(f'Speed_Trace_{left}to{right}s.png', format='png', dpi=900)
plt.show()

# Done!